In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import random
from random import sample
import numpy as np
import time
from sklearn import svm
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation
from sklearn import grid_search

In [2]:
# SVM CLASSIFICATION FUNCTIONS
def randomSplit(X,y):
    accuracy = np.array([])
    # do 20 random splits
    for i in range(2):
        # leave 20% out for testing
        skf = cross_validation.StratifiedKFold(y,n_folds=10,shuffle=True) 
        
        for cv_i,test_i in skf:
            print "Train index: " , cv_i
            print "Test index: " , test_i
            train_X = X[cv_i]
            train_y = y[cv_i]
            # do training here
            clf = gridSearchSVM(train_X,train_y)

            # do testing here
            test_X = X[test_i]
            test_y = y[test_i]
            prediction = clf.predict(test_X)
            print clf.best_params_
            # record performance
            foo = performance(prediction, test_y)
            print "Accuracy: " , foo
            accuracy = np.append(accuracy, foo)
            break
        
    mean_acc = np.mean(accuracy)
    return mean_acc


def gridSearchSVM(X,y):
    #parameters = {'kernel':('linear','rbf'), 'C':[1, 2, 3, 5, 10, 13, 15,20]}
    parameters = {'kernel':('linear','rbf'), 'C':[1, 2, 3, 5]}
    svr  = svm.SVC()
    clf = grid_search.GridSearchCV(svr, parameters, cv = 5, n_jobs = 4, verbose=1)
    clf.fit(X,y)
    return clf

def performance(prediction, target):
    acc = metrics.accuracy_score(target, prediction, normalize=True)
    return acc

In [3]:
# DATA LOADING FUNCTIONS
def loadYaleData():
    X = np.matrix(scipy.io.loadmat('yalefacesFFT.mat')['DATA'])
    individuals = 15; 
    picsPerInd = 11;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)

def loadOlivettiData():
    X = np.matrix(scipy.io.loadmat('olivettifacesFFT.mat')['DATA'])
    individuals = 40; 
    picsPerInd = 10;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)

In [27]:
# TRANFORMATION FUNCTIONS

def removePhase(V):
    return V[0:V.shape[0]/2].reshape(V.shape[0]/2);

def recFilter(w,h,fw,fh):
    h = h + 1
    F = np.zeros((h,w));
    for i in range((w-1)/2+1 - fw/2, (w-1)/2+1 + fw/2 - 1):
        for j in range(h - fh - 1, h):
            F[j,i] = 1
    return np.reshape(F,(w*h),order='F')

def triFilter(w,h,fw,fh):
    h = h + 1
    F = np.zeros((h,w));
    for j in range(h - fh -1, h):
        span = (j - (h - fh)) * (fw/2)/fh;
        for i in range((w-1)/2+1 - span, (w-1)/2+1 + span - 1):
            F[j,i] = 1
    return np.reshape(F,(w*h),order='F')

In [5]:
# RECONSTRUCTION FUNCTIONS

# Reconstruct an image from the Yale database 
# based on the all frequecies and phases
def getYaleface(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

def getYaleFFTImg(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0); 
    return np.log(FFTimg + 1);

# Reconstruct an image from the Olivetti database 
# based on the all frequecies and phases
def getOlivettiface(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

def getOlivettiFFTImg(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0); 
    return np.log(FFTimg + 1);

def addZeroPhase(V,):
    F = np.zeros((V.shape[0]*2))
    # add magnitude 
    F[0:V.shape[0]] = V[0:V.shape[0]].reshape(V.shape[0]);
    # add original phase
    #F[V.shape[0]:] = X[V.shape[0]:, i].reshape(V.shape[0]);
    # add zero phase
    F[V.shape[0]:] = 0;
    return F.reshape(F.shape[0]);

In [22]:
# DISPLAY FUNCTIONS
# h,w are height and width of the individual images, nh,nw are the dimensions of the 
# collage interms of images 
def makeCollage(h, w, nh, nw, DATA, randImg, getImage):
    
    border = 10
    collage = np.zeros((h*nh+border*(nh+1),w*nw+border*(nw+1)));
    #randImg = sample(xrange(DATA.shape[1]), nw*nh)

    for i in range(0,nw):
        for j in range(0,nh):
            img = getImage(DATA[:,randImg[nw*i + j]])
            collage[(i+1)*border-1+i*h:(i+1)*h+(i+1)*border-1, (j+1)*border+j*w-1:(j+1)*border-1+(j+1)*w] = img
    
    return collage
    

In [111]:
# BASELINE CLASSIFICATION
(X,Y) = loadOlivettiData()
print randomSplit(np.transpose(X),Y)

In [25]:
(X,Y) = loadYaleData() 
plt.imshow(makeCollage(63, 63, 10, 10, X, random.sample(range(Y.shape[0]), 100), getYaleface));
plt.gray()
plt.show()

(X,Y) = loadOlivettiData()
plt.imshow(makeCollage(63, 63, 10, 10, X, random.sample(range(Y.shape[0]), 100), getYaleface));
plt.gray()
plt.show()

In [7]:
# NO PHASE CLASSIFICATION
yaleHeight = 243
yaleWidth = 319
olivettHeight = 63
olivettWidth = 63

(X,Y) = loadOlivettiData()


xNoPhase = np.zeros([X.shape[0]/2, X.shape[1]])

for i in range(0,X.shape[1]):
    xNoPhase[:,i] = removePhase(X[:,i])[:]

xZeroPhase = np.zeros([X.shape[0], X.shape[1]])
for i in range(0,X.shape[1]):
    xZeroPhase[:,i] = addZeroPhase(xNoPhase[:,i])[:]

print X.shape
print xNoPhase.shape
print xZeroPhase.shape

randImgs = sample(xrange(X.shape[1]), 4)
makeCollage(olivettHeight, olivettWidth, 1, 1, X, randImgs, getOlivettiface)
makeCollage(olivettHeight, olivettWidth, 1, 1, xZeroPhase, randImgs, getOlivettiface)

#print randomSplit(np.transpose(xNoPhase),Y)

(4032, 400)
(2016, 400)
(4032, 400)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [31]:
# Make Filtered Datasets
height = 63
width = 63

for w in range(5, 40, 5):
    (X,Y) = loadOlivettiData()
    F = triFilter(width, (height-1)/2, w, w)
    randImgs = [0, 72, 84, 9, 111]

    for i in range(0, X.shape[1]):
        for j in range(0, F.shape[0]):
            X[j,i] = X[j,i] * F[j]

    #print np.count_nonzero(X[0:F.shape[0]+1,0])
    np.save("olivettiF_R_" + str(w) + "_" + str(w), X)
    c = makeCollage(height, width, 1, 1, X, randImgs, getOlivettiFFTImg)
    scipy.misc.imsave('oFFT_R_' + str(w) + "_" + str(w) + '.jpg', c)
    c = makeCollage(height, width, 1, 1, X, randImgs, getOlivettiface)
    scipy.misc.imsave('oRecon_R_' + str(w) + "_" + str(w) + '.jpg', c)

C:\Users\AI\Anaconda\lib\site-packages\IPython\kernel\__main__.py:13: ComplexWarning: Casting complex values to real discards the imaginary part


In [112]:
# Run Classification on Filtered Data (Rectangle)
accuracy_phase = np.array([])
dim_phase = np.array([]) 
accuracy_nophase = np.array([])
dim_nophase = np.array([]) 

# load the labels 
(X,Y) = loadYaleData()

for w in range(10, 130, 10):
    # load filtered data 
    X = np.load("./Filters/FilteredData/yaleF_R_" + str(w) + "_" + str(w) + ".npy")
        
    X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
    for i in range(0,X.shape[1]):
        X_nophase[:,i] = removePhase(X[:,i])[:]
    
    
    accuracy_phase = np.append(accuracy_phase, randomSplit(np.transpose(X),Y))
    print accuracy_phase[-1]
    
    dim_phase = np.append(dim_phase, w*w + X.shape[0]/2) 
    print dim_phase[-1]
    
    accuracy_nophase = np.append(accuracy_nophase, randomSplit(np.transpose(X_nophase),Y))
    print accuracy_nophase[-1]
    
    dim_nophase = np.append(dim_nophase, w*w)
    print dim_nophase[-1]

np.savetxt("Y_R_acc_phase", accuracy_phase)
np.savetxt("Y_R_dim_phase", dim_phase)
np.savetxt("Y_R_acc_nophase", accuracy_nophase)
np.savetxt("Y_R_dim_phase", dim_nophase)


In [8]:
# Run Classification on Filtered Data (Triangle)
accuracy_phase = np.array([])
dim_phase = np.array([]) 
accuracy_nophase = np.array([])
dim_nophase = np.array([]) 

# load the labels 
(X,Y) = loadOlivettiData()

for w in range(10, 70, 10):
    # load filtered data 
    X = np.load("./Filters/FilteredData/olivettiF_T_" + str(w) + "_" + str(w) + ".npy")
    
    #c = makeCollage(olivettHeight, olivettWidth, 1, 1, X, randImgs, getOlivettiFFTImg)
    #plt.imshow(c)
    #plt.show()
    
    X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
    for i in range(0,X.shape[1]):
        X_nophase[:,i] = removePhase(X[:,i])[:]
    
    
    accuracy_phase = np.append(accuracy_phase, randomSplit(np.transpose(X),Y))
    print accuracy_phase[-1]
    
    dim_phase = np.append(dim_phase, np.count_nonzero(X[0:X.shape[0]/2,0]) + X.shape[0]/2)
    print dim_phase[-1]
    
    accuracy_nophase = np.append(accuracy_nophase, randomSplit(np.transpose(X_nophase),Y))
    print accuracy_nophase[-1]
    
    dim_nophase = np.append(dim_nophase, np.count_nonzero(X[0:X.shape[0]/2,0]))
    print dim_nophase[-1]

np.savetxt("O_T_acc_phase", accuracy_phase)
np.savetxt("O_T_dim_phase", dim_phase)
np.savetxt("O_T_acc_nophase", accuracy_nophase)
np.savetxt("O_T_dim_nophase", dim_nophase)


Train index:  [  0   1   2   3   4   5   6   8   9  10  11  12  13  14  15  16  17  18
  20  21  22  23  24  26  27  28  29  30  31  32  34  35  36  37  38  39
  40  41  42  44  45  46  47  48  49  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  66  67  68  70  71  72  73  74  75  77  78  79
  80  81  82  83  84  86  87  88  89  90  91  92  93  94  95  96  97  98
 100 101 102 103 104 105 106 107 108 110 111 112 113 114 115 116 117 119
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 139
 141 142 143 144 145 146 147 148 149 151 152 153 154 155 156 157 158 159
 160 162 163 164 165 166 167 168 169 170 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 187 188 189 190 191 192 193 194 196 197 198 199
 200 201 202 204 205 206 207 208 209 210 211 213 214 215 216 217 218 219
 220 222 223 224 225 226 227 228 229 230 231 233 234 235 236 237 238 239
 240 241 242 243 244 245 247 248 249 250 251 252 253 254 256 257 258 259
 260 261 262 263 264 265 267 268 269 

[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done  34 out of  40 | elapsed:   16.0s remaining:    2.8s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   19.3s finished


{'kernel': 'linear', 'C': 1}
Accuracy:  0.925
Train index:  [  0   1   2   3   4   5   6   7   8  10  11  12  13  14  15  17  18  19
  20  21  22  23  24  25  26  27  28  30  31  32  34  35  36  37  38  39
  40  42  43  44  45  46  47  48  49  50  51  53  54  55  56  57  58  59
  60  61  62  64  65  66  67  68  69  71  72  73  74  75  76  77  78  79
  80  81  82  83  84  85  86  87  88  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 107 108 109 110 111 112 114 115 116 117 118 119
 120 121 122 123 124 125 126 128 129 130 131 132 133 134 135 136 137 139
 140 141 142 143 144 145 147 148 149 150 151 152 153 154 155 156 158 159
 160 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 178 179
 180 181 183 184 185 186 187 188 189 190 191 192 193 194 195 196 198 199
 200 201 202 203 204 205 206 207 209 210 211 212 213 214 215 216 217 218
 221 222 223 224 225 226 227 228 229 230 231 233 234 235 236 237 238 239
 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256

[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done  34 out of  40 | elapsed:   15.9s remaining:    2.8s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   19.2s finished


{'kernel': 'linear', 'C': 1}
Accuracy:  0.95
0.9375
2048.0
Train index:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18  19
  20  21  22  23  24  25  26  27  29  30  31  32  33  34  35  36  37  38
  40  41  42  43  45  46  47  48  49  50  51  52  53  54  55  57  58  59
  60  61  62  63  65  66  67  68  69  70  71  72  74  75  76  77  78  79
  80  81  82  83  84  85  87  88  89  90  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 109 111 112 113 114 115 116 117 118 119
 120 121 122 124 125 126 127 128 129 130 131 132 133 134 135 137 138 139
 141 142 143 144 145 146 147 148 149 150 151 153 154 155 156 157 158 159
 160 161 163 164 165 166 167 168 169 170 171 172 174 175 176 177 178 179
 180 181 182 183 184 186 187 188 189 190 191 192 193 194 195 196 197 198
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 219
 220 221 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238
 240 241 242 244 245 246 247 248 249 250 251 253 25

...

{'kernel': 'linear', 'C': 1}
Accuracy:  0.975
0.9625
3360.0
Train index:  [  0   1   2   4   5   6   7   8   9  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  28  29  30  31  32  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  49  50  51  52  53  54  55  56  58  59
  60  61  62  63  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  80  81  82  83  84  85  86  88  89  90  91  92  94  95  96  97  98  99
 100 101 102 103 104 105 107 108 109 110 111 112 113 114 115 117 118 119
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 139
 140 142 143 144 145 146 147 148 149 151 152 153 154 155 156 157 158 159
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 178 179
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 200 201 202 203 204 206 207 208 209 210 211 212 213 215 216 217 218 219
 221 222 223 224 225 226 227 228 229 230 231 232 234 235 236 237 238 239
 240 241 242 243 244 245 246 247 249 250 251 252 2

[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  34 out of  40 | elapsed:    8.7s remaining:    1.5s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   10.3s finished


{'kernel': 'linear', 'C': 1}
Accuracy:  1.0
Train index:  [  0   1   2   3   4   5   6   7   9  10  11  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  28  29  30  31  32  33  34  36  37  38  39
  40  41  42  43  44  45  46  47  49  50  51  52  53  54  55  56  58  59
  60  61  62  64  65  66  67  68  69  71  72  73  74  75  76  77  78  79
  80  82  83  84  85  86  87  88  89  90  91  92  93  94  95  97  98  99
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 118 119
 120 121 122 123 124 125 126 128 129 130 131 132 133 134 136 137 138 139
 140 141 142 143 144 146 147 148 149 150 151 152 153 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 170 171 173 174 175 176 177 178 179
 180 182 183 184 185 186 187 188 189 190 191 192 193 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 210 211 212 213 214 215 217 218 219
 220 221 223 224 225 226 227 228 229 230 231 232 233 235 236 237 238 239
 240 241 242 243 244 245 247 248 249 250 251 252 253 254 255 256 2

[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  34 out of  40 | elapsed:    8.7s remaining:    1.5s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   10.4s finished


{'kernel': 'linear', 'C': 1}
Accuracy:  1.0
1.0
1344.0


In [22]:
accuracy_phase = np.loadtxt("Y_R_acc_phase")
dim_phase = np.loadtxt("Y_R_dim_phase")
accuracy_nophase = np.loadtxt("Y_R_acc_nophase")
dim_nophase = np.loadtxt("Y_R_dim_nophase")

plt.title("Yale Rectanular Filter With Phase")
plt.xlabel("Dimension")
plt.ylabel("Mean Accuracy")
plt.scatter(dim_phase, accuracy_phase)
plt.show()

plt.title("Yale Rectanular Filter Without Phase")
plt.xlabel("Dimension")
plt.ylabel("Mean Accuracy")
plt.scatter(dim_nophase, accuracy_nophase)
plt.show()

ValueError: sample larger than population